In [118]:
import pandas as pd
#reading the csv file and replacing all the 0 and '[]' with Nan
data = pd.read_csv("tmdb_5000_movies.csv", na_values=[0,'[]'])
len(data.columns)

data.shape

(4803, 20)

In [119]:
#dropping columns which are not useful for analysis
data.drop(['homepage','id','keywords','original_title','overview','tagline'], axis = 1, inplace=True)
data.shape

(4803, 14)

In [120]:
#dropping rows with more than 3 null values
data = data.dropna(thresh=11)
data.shape

(4633, 14)

In [121]:
#dropping rows with no production companies 
data = data.dropna(subset=['production_companies'])
data.shape

(4428, 14)

In [122]:
#creating a dataframe which stores the average budget of all the poduction companies

import pandas as pd
import json
import math
d = {}

def company_average(budget, companies):
    global d
    if math.isnan(budget):
        return
    
    data = json.loads(companies)
    per_company = budget/len(companies)
    
    for company in data:
        name = company['name']
        if name in d.keys():
            d[name]['total'] +=per_company
            d[name]['number'] +=1
            d[name]['total'] =d[name]['total']/d[name]['number']
        else:
            d[name]={'total':per_company, 'average': per_company, 'number':1}
    
for i in (data.index):
    company_average(data.loc[i,'budget'],data.loc[i,'production_companies'])
    
companies = pd.DataFrame(d).transpose()
companies


total       average  number
Ingenious Film Partners                    967.629818  1.167488e+06    27.0
Twentieth Century Fox Film Corporation     564.221068  1.167488e+06   209.0
Dune Entertainment                        1349.540852  1.167488e+06    56.0
Lightstorm Entertainment                171555.478802  1.167488e+06     5.0
Walt Disney Pictures                       355.796157  2.127660e+06   105.0
...                                               ...           ...     ...
Strand Releasing                           642.857143  6.428571e+02     1.0
Dreamland Productions                      260.869565  2.608696e+02     1.0
Park Films                                   0.158537  1.585366e-01     1.0
Camera Stylo Films                           0.158537  1.585366e-01     1.0
Daiei Studios                              526.315789  5.263158e+02     1.0

[4190 rows x 3 columns]

In [123]:
#using the companies dataframe to fill the rows with null budget with the sum of the averages of the companies' budget

import json

#number of null budget values before filling with average = 754
print(data[data['budget'].isna()].shape)

def get_average(data):
    
    comp = json.loads(data)
    budget = 0
    for i in comp:
        if i['name'] in companies.index:
            budget += companies.loc[i['name'],'average']
        else:
            return
    return budget


for i in data[data['budget'].isna()].index:
    average = get_average(data.loc[i,'production_companies'])
    data.loc[i,'budget']= average

#number of null budget values after filling with average = 477    
data[data['budget'].isna()].shape

(754, 14)


(477, 14)